<a href="https://colab.research.google.com/github/CharlesPrado23/Notebooks/blob/main/Semana8/Semana8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Content-based Recommender Systems

In [1]:
import pandas as pd
import numpy as np


In [37]:
df_title_basics = pd.read_csv('/content/IMDb movies.csv')
df_title_basics.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [38]:
index = df_title_basics['year'].str.match('[A-Z]') 
index
kx=[]
for i in range(len(index)): 
    if index[i] == True:
        kx.append(i)
kx      
for i in range(len(kx)):
    df_title_basics = df_title_basics.drop(index = kx[i])


In [41]:
df_title_basics[["year"]] = df_title_basics[["year"]].apply(pd.to_numeric)
df_temp = df_title_basics[(df_title_basics['year'] > 1950) & (df_title_basics['country'] == 'USA') & (df_title_basics['avg_vote'] > 7.5) ]
movies_df = df_temp[['original_title','genre']]

In [42]:
movies_df.tail()

,original_title,genre
85093,Dark Waters,"Biography, Drama, History"
85141,Michael Jackson: Smooth Criminal (II),"Action, Crime, Fantasy"
85151,Breaking Fast,"Comedy, Drama"
85518,The Guy Who Didn't Like Musicals,"Comedy, Musical"
85744,Apollo: Missions to the Moon,History


In [43]:
movies_df.duplicated().sum()

0

In [44]:
movies_df = movies_df.drop_duplicates()

In [45]:
movies_df['genre'] = movies_df.genre.str.split(',')
movies_df

,original_title,genre
6752,Ace in the Hole,"[Drama, Film-Noir]"
6793,The Day the Earth Stood Still,"[Drama, Sci-Fi]"
6798,Detective Story,"[Crime, Drama, Film-Noir]"
6979,A Place in the Sun,"[Drama, Romance]"
7024,Strangers on a Train,"[Crime, Film-Noir, Thriller]"
...,...,...
85093,Dark Waters,"[Biography, Drama, History]"
85141,Michael Jackson: Smooth Criminal (II),"[Action, Crime, Fantasy]"
85151,Breaking Fast,"[Comedy, Drama]"
85518,The Guy Who Didn't Like Musicals,"[Comedy, Musical]"


In [46]:
moviesWithGenres_df = movies_df.copy()
for index, row in movies_df.iterrows():
    for genre in row['genre']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,original_title,genre,Drama,Film-Noir,Sci-Fi,Crime,Drama,Romance,Thriller,Adventure,Western,Music,Comedy,Musical,War,Mystery,Mystery,Comedy,Family,Horror,Horror,Romance,History,Action,Biography,Adventure,Biography,Sport,Crime,Fantasy,Sci-Fi,Animation,Thriller,Music,Western,Family,Action,Musical,Fantasy,History
6752,Ace in the Hole,"[Drama, Film-Noir]",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6793,The Day the Earth Stood Still,"[Drama, Sci-Fi]",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6798,Detective Story,"[Crime, Drama, Film-Noir]",0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6979,A Place in the Sun,"[Drama, Romance]",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7024,Strangers on a Train,"[Crime, Film-Noir, Thriller]",0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#moviesWithGenres_df[moviesWithGenres_df['original_title'].str.contains("Matrix")]

In [47]:
userInput = [
            {'original_title':'A Beautiful Mind', 'rating':7.0},
            {'original_title':'The Irishman', 'rating':3.5},
            {'original_title':'The Godfather', 'rating':7.0},
            {'original_title':"Captain America: Civil War", 'rating':5},
            {'original_title':'Star Wars', 'rating':8.5},
            {'original_title':'The Matrix', 'rating':8.5} 
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,original_title,rating
0,A Beautiful Mind,7.0
1,The Irishman,3.5
2,The Godfather,7.0
3,Captain America: Civil War,5.0
4,Star Wars,8.5
5,The Matrix,8.5


In [48]:
inputId = movies_df[movies_df['original_title'].isin(inputMovies['original_title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,original_title,genre,rating
0,The Godfather,"[Crime, Drama]",7.0
1,Star Wars,"[Action, Adventure, Fantasy]",8.5
2,The Matrix,"[Action, Sci-Fi]",8.5
3,A Beautiful Mind,"[Biography, Drama]",7.0
4,The Irishman,"[Biography, Crime, Drama]",3.5
5,Captain America: Civil War,"[Action, Adventure, Sci-Fi]",5.0


In [49]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['original_title'].isin(inputMovies['original_title'].tolist())]
userMovies

,original_title,genre,Drama,Film-Noir,Sci-Fi,Crime,Drama,Romance,Thriller,Adventure,Western,Music,Comedy,Musical,War,Mystery,Mystery,Comedy,Family,Horror,Horror,Romance,History,Action,Biography,Adventure,Biography,Sport,Crime,Fantasy,Sci-Fi,Animation,Thriller,Music,Western,Family,Action,Musical,Fantasy,History
15528,The Godfather,"[Crime, Drama]",0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18216,Star Wars,"[Action, Adventure, Fantasy]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32229,The Matrix,"[Action, Sci-Fi]",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39274,A Beautiful Mind,"[Biography, Drama]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56680,The Irishman,"[Biography, Crime, Drama]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71476,Captain America: Civil War,"[Action, Adventure, Sci-Fi]",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('original_title', 1).drop('genre', 1)
userGenreTable

,Drama,Film-Noir,Sci-Fi,Crime,Drama,Romance,Thriller,Adventure,Western,Music,Comedy,Musical,War,Mystery,Mystery,Comedy,Family,Horror,Horror,Romance,History,Action,Biography,Adventure,Biography,Sport,Crime,Fantasy,Sci-Fi,Animation,Thriller,Music,Western,Family,Action,Musical,Fantasy,History
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
inputMovies['rating']

0    7.0
1    8.5
2    8.5
3    7.0
4    3.5
5    5.0
Name: rating, dtype: float64

In [52]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Drama          0.0
 Film-Noir     0.0
 Sci-Fi       13.5
Crime          7.0
 Drama        17.5
 Romance       0.0
 Thriller      0.0
Adventure      0.0
 Western       0.0
 Music         0.0
Comedy         0.0
 Musical       0.0
 War           0.0
Mystery        0.0
 Mystery       0.0
 Comedy        0.0
 Family        0.0
 Horror        0.0
Horror         0.0
Romance        0.0
 History       0.0
Action        22.0
Biography     10.5
 Adventure    13.5
 Biography     0.0
 Sport         0.0
 Crime         3.5
 Fantasy       8.5
Sci-Fi         0.0
Animation      0.0
Thriller       0.0
Music          0.0
Western        0.0
Family         0.0
 Action        0.0
Musical        0.0
Fantasy        0.0
History        0.0
dtype: float64

In [53]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['original_title'])
#And drop the unnecessary information
genreTable = genreTable.drop('original_title', 1).drop('genre', 1)
genreTable

,Drama,Film-Noir,Sci-Fi,Crime,Drama,Romance,Thriller,Adventure,Western,Music,Comedy,Musical,War,Mystery,Mystery,Comedy,Family,Horror,Horror,Romance,History,Action,Biography,Adventure,Biography,Sport,Crime,Fantasy,Sci-Fi,Animation,Thriller,Music,Western,Family,Action,Musical,Fantasy,History
original_title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ace in the Hole,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Day the Earth Stood Still,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Detective Story,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Place in the Sun,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Strangers on a Train,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dark Waters,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Michael Jackson: Smooth Criminal (II),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Breaking Fast,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df

original_title
Ace in the Hole                          0.000000
The Day the Earth Stood Still            0.140625
Detective Story                          0.255208
A Place in the Sun                       0.000000
Strangers on a Train                     0.072917
                                           ...   
Dark Waters                              0.291667
Michael Jackson: Smooth Criminal (II)    0.354167
Breaking Fast                            0.182292
The Guy Who Didn't Like Musicals         0.000000
Apollo: Missions to the Moon             0.000000
Length: 605, dtype: float64

In [55]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head(50)

original_title
The Last of the Mohicans                                  0.552083
Avengers: Endgame                                         0.552083
Logan                                                     0.552083
Star Trek: First Contact                                  0.552083
The Avengers                                              0.510417
Captain America: The Winter Soldier                       0.510417
Serenity                                                  0.510417
Jurassic Park                                             0.510417
Star Trek II: The Wrath of Khan                           0.510417
Rogue One                                                 0.510417
Star Trek Into Darkness                                   0.510417
Star Wars: Episode VII - The Force Awakens                0.510417
Captain America: Civil War                                0.510417
Avengers: Infinity War                                    0.510417
The Crow                                       